In [39]:
import pandas as pd
import numpy as np

results = pd.read_csv('results.csv', parse_dates=['date'])
shootouts = pd.read_csv('shootouts.csv', parse_dates=['date'])
populations = pd.read_csv('Global Nations Economy -Population.csv')

In [2]:
modern_date = '2010-01-01'
mod_results = results[results['date'] >= modern_date].reset_index(drop=True)   #shows only results since input date
mod_shootouts = shootouts[shootouts['date'] >= modern_date].reset_index(drop=True)   #shows only shootouts since input date

mod_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,2010-01-02,Iran,North Korea,1,0,Friendly,Doha,Qatar,True
1,2010-01-02,Qatar,Mali,0,0,Friendly,Doha,Qatar,False
2,2010-01-02,Syria,Zimbabwe,6,0,Friendly,Kuala Lumpur,Malaysia,True
3,2010-01-02,Yemen,Tajikistan,0,1,Friendly,Sana'a,Yemen,False
4,2010-01-03,Angola,Gambia,1,1,Friendly,Vila Real de Santo António,Portugal,True


In [3]:
mod_shootouts.head()

,date,home_team,away_team,winner
0,2010-01-25,Zambia,Nigeria,Nigeria
1,2010-06-29,Paraguay,Japan,Paraguay
2,2010-07-02,Uruguay,Ghana,Uruguay
3,2011-01-25,Japan,South Korea,Japan
4,2011-06-18,Costa Rica,Honduras,Honduras


In [4]:
#merges two dfs into one, puts NaN in 'winner' for games without shootout
mod_full = mod_results.merge(mod_shootouts, how='outer', on=['date', 'home_team', 'away_team'])
mod_full.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner
0,2010-01-02,Iran,North Korea,1,0,Friendly,Doha,Qatar,True,NaN
1,2010-01-02,Qatar,Mali,0,0,Friendly,Doha,Qatar,False,NaN
2,2010-01-02,Syria,Zimbabwe,6,0,Friendly,Kuala Lumpur,Malaysia,True,NaN
3,2010-01-02,Yemen,Tajikistan,0,1,Friendly,Sana'a,Yemen,False,NaN
4,2010-01-03,Angola,Gambia,1,1,Friendly,Vila Real de Santo António,Portugal,True,NaN


In [5]:
#puts winning country in winner column, or 'Tie' otherwise, and puts 'Home', 'Away', or 'Tie' in winner_loc column
for x in range(len(mod_full)):
    if (pd.isna(mod_full['winner'][x])):   #avoids replacing values from shootouts
        if (mod_full['home_score'][x] > mod_full['away_score'][x]):
            mod_full.at[x, 'winner'] = mod_full['home_team'][x]
        else:
            if (mod_full['home_score'][x] < mod_full['away_score'][x]):
                mod_full.at[x, 'winner'] = mod_full['away_team'][x]
            else:
                mod_full.at[x, 'winner'] = 'Tie'
mod_full.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner
0,2010-01-02,Iran,North Korea,1,0,Friendly,Doha,Qatar,True,Iran
1,2010-01-02,Qatar,Mali,0,0,Friendly,Doha,Qatar,False,Tie
2,2010-01-02,Syria,Zimbabwe,6,0,Friendly,Kuala Lumpur,Malaysia,True,Syria
3,2010-01-02,Yemen,Tajikistan,0,1,Friendly,Sana'a,Yemen,False,Tajikistan
4,2010-01-03,Angola,Gambia,1,1,Friendly,Vila Real de Santo António,Portugal,True,Tie


In [6]:
mod_full['winner_loc'] = ''
for x in range(len(mod_full)):
    if (mod_full['home_score'][x] > mod_full['away_score'][x]):
        mod_full.at[x, 'winner_loc'] = 'Home'
    else:
        if (mod_full['home_score'][x] < mod_full['away_score'][x]):
            mod_full.at[x, 'winner_loc'] = 'Away'
        else:
            mod_full.at[x, 'winner_loc'] = 'Tie'
mod_full.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner,winner_loc
0,2010-01-02,Iran,North Korea,1,0,Friendly,Doha,Qatar,True,Iran,Home
1,2010-01-02,Qatar,Mali,0,0,Friendly,Doha,Qatar,False,Tie,Tie
2,2010-01-02,Syria,Zimbabwe,6,0,Friendly,Kuala Lumpur,Malaysia,True,Syria,Home
3,2010-01-02,Yemen,Tajikistan,0,1,Friendly,Sana'a,Yemen,False,Tajikistan,Away
4,2010-01-03,Angola,Gambia,1,1,Friendly,Vila Real de Santo António,Portugal,True,Tie,Tie


In [19]:
a = set(mod_full['home_team'])
b = set(mod_full['away_team'])
all_teams = a.union(b)   #all teams in dataset
eu_teams = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland',
            'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
            'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']

In [24]:
mod_eu = mod_full[(mod_full['home_team'].isin(eu_teams)) | (mod_full['away_team'].isin(eu_teams))]
mod_eu.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner,winner_loc
41,2010-01-18,Finland,South Korea,0,2,Friendly,Málaga,Spain,True,South Korea,Away
47,2010-01-20,Oman,Sweden,0,1,Friendly,Muscat,Oman,False,Sweden,Away
48,2010-01-20,Thailand,Poland,1,3,King's Cup,Nakhon Ratchasima,Thailand,False,Poland,Away
52,2010-01-22,Latvia,South Korea,0,1,Friendly,Málaga,Spain,True,South Korea,Away
53,2010-01-23,Poland,Singapore,6,1,King's Cup,Nakhon Ratchasima,Thailand,True,Poland,Home


In [41]:
populations.head()

,Country Name,2018,2017,2016,2015,2014
0,China,"1,39,27,30,000","1,38,63,95,000","1,37,86,65,000","1,37,12,20,000","1,36,42,70,000"
1,India,"1,35,26,42,283","1,33,86,76,779","1,32,45,17,250","1,31,01,52,392","1,29,56,00,768"
2,United States,"32,68,38,199","32,51,22,128","32,30,71,755","32,07,38,994","31,83,86,329"
3,Indonesia,"26,76,70,549","26,46,50,969","26,15,56,386","25,83,83,257","25,51,28,076"
4,Pakistan,"21,22,28,288","20,79,06,210","20,36,31,356","19,94,26,953","19,53,05,012"


In [95]:
migrant_countries = ['Hungary', 'Sweden', 'Austria', 'Norway', 'Finland', 'Germany', 'Switzerland', 'Luxembourg', 'Malta', 
                    'Denmark', 'Belgium', 'Bulgaria', 'Netherlands', 'Cyprus', 'Italy', 'France', 'Greece', 'Ireland']
migrant_numbers = [1770, 1600, 1000, 590, 590, 540, 460, 420, 390, 370, 350, 280, 250, 250, 140, 
                              110, 100, 70]
migrant_numbers = map(lambda x: x/100000, migrant_numbers)
migrant_2015 = pd.DataFrame([migrant_numbers, migrant_countries], index={'Proportion', 'Country Name'}).transpose()
eu_populations_2015 = populations[populations['Country Name'].isin(migrant_countries)].replace(',', '', regex=True)
eu_migrants = migrant_2015.merge(eu_populations_2015, how='outer', on='Country Name')
eu_migrants['2015'] = eu_migrants['2015'].apply(pd.to_numeric)
eu_migrants['immigrants_2015'] = eu_migrants['Proportion']*eu_migrants['2015']


print(eu_migrants)

   Proportion Country Name      2018      2017      2016      2015      2014  \
0      0.0177      Hungary   9775564   9787966   9814023   9843028   9866468   
1       0.016       Sweden  10175214  10057698   9923085   9799186   9696110   
2        0.01      Austria   8840521   8797566   8736668   8642699   8546356   
3      0.0059       Norway   5311916   5276968   5234519   5188607   5137232   
4      0.0059      Finland   5515525   5508214   5495303   5479531   5461512   
5      0.0054      Germany  82905782  82657002  82348669  81686611  80982500   
6      0.0046  Switzerland   8514329   8451840   8373338   8282396   8188649   
7      0.0042   Luxembourg    607950    596336    582014    569604    556319   
8      0.0039        Malta    484630    467999    455356    445053    434558   
9      0.0037      Denmark   5793636   5764980   5728010   5683483   5643475   
10     0.0035      Belgium  11427054  11375158  11331422  11274196  11209057   
11     0.0028     Bulgaria   7025037   7